In [1]:
import threading
import multiprocessing
import requests
from urllib.parse import urlparse

def download_content(url):
    response = requests.get(url)
    return response.content

class DownloadManager:
    def __init__(self, max_threads, max_processes):
        self.max_threads = max_threads
        self.max_processes = max_processes
        self.downloads = []

    def add_download(self, url, filename):
        parsed_url = urlparse(url)
        if parsed_url.scheme == 'http' or parsed_url.scheme == 'https':
            download = ThreadingDownloader(url, filename)
        elif parsed_url.scheme == 'ftp':
            download = MultiprocessingDownloader(url, filename)
        else:
            raise ValueError("Unsupported URL scheme.")
        self.downloads.append(download)

    def start_downloads(self):
        for download in self.downloads:
            download.start_download()

    def wait_for_downloads(self):
        for download in self.downloads:
            download.wait_for_completion()

class Download:
    def __init__(self, url, filename):
        self.url = url
        self.filename = filename
        self.is_complete = False

    def start_download(self):
        raise NotImplementedError

    def save_file(self, content):
        with open(self.filename, 'wb') as file:
            file.write(content)

    def download_complete(self):
        self.is_complete = True

class ThreadingDownloader(Download):
    def start_download(self):
        download_thread = threading.Thread(target=self._download)
        download_thread.start()

    def _download(self):
        content = download_content(self.url)
        self.save_file(content)
        self.download_complete()

    def wait_for_completion(self):
        pass

class MultiprocessingDownloader(Download):
    def start_download(self):
        download_process = multiprocessing.Process(target=self._download)
        download_process.start()

    def _download(self):
        content = download_content(self.url)
        self.save_file(content)
        self.download_complete()

    def wait_for_completion(self):
        pass

if __name__ == "__main__":
    manager = DownloadManager(max_threads=3, max_processes=2)

    manager.add_download("http://example.com/file1.txt", "file1.txt")
    manager.add_download("ftp://example.com/file2.txt", "file2.txt")
    manager.add_download("http://example.com/file3.txt", "file3.txt")

    manager.start_downloads()
    manager.wait_for_downloads()

    print("All downloads completed!")


All downloads completed!
